<a href="https://colab.research.google.com/github/adishukla2009/EVA4/blob/Session4/Ninety_nine_point_four_six.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3) #28 26 3
        self.Batch1 = nn.BatchNorm2d(16)
        self.Drop1 = nn.Dropout(0.1)

        self.conv2 = nn.Conv2d(16, 16, 3) #26 24 5
        self.Batch2 = nn.BatchNorm2d(16)
        self.Drop2 = nn.Dropout(0.1)

        self.conv3 = nn.Conv2d(16,16,1) #24 24 5
        self.pool1 = nn.MaxPool2d(2, 2) #24 12 10

        self.conv4 = nn.Conv2d(16, 16, 3) #12 10 12
        self.Batch4 = nn.BatchNorm2d(16)
        self.Drop4 = nn.Dropout(0.1)

        self.conv5 = nn.Conv2d(16, 16, 3) #10 8 14
        self.Batch5 = nn.BatchNorm2d(16)
        self.Drop5 = nn.Dropout(0.1)

        self.conv6 = nn.Conv2d(16, 16, 3) #8 6 16
        self.Batch6 = nn.BatchNorm2d(16)
        self.Drop6 = nn.Dropout(0.1)

        self.conv7 = nn.Conv2d(16, 32, 3) #6 4 18
        self.Batch7 = nn.BatchNorm2d(32)
        self.Drop7 = nn.Dropout(0.1)

        self.conv8 = nn.Conv2d(32 ,10, 4)
 
    def forward(self, x):
        x= F.relu(self.conv1(x))
        x=self.Batch1(x)
        x=self.Drop1(x)

        x= F.relu(self.conv2(x))
        x=self.Batch2(x)
        x=self.Drop2(x)

        x=self.pool1(F.relu(self.conv3(x)))

        x= F.relu(self.conv4(x))
        x=self.Batch4(x)
        x=self.Drop4(x)

        x= F.relu(self.conv5(x))
        x=self.Batch5(x)
        x=self.Drop5(x)

        x= F.relu(self.conv6(x))
        x=self.Batch6(x)
        x=self.Drop6(x)

        x= F.relu(self.conv7(x))
        x=self.Batch7(x)
        x=self.Drop7(x)

        x=self.conv8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,320
       BatchNorm2d-5           [-1, 16, 24, 24]              32
           Dropout-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 16, 24, 24]             272
         MaxPool2d-8           [-1, 16, 12, 12]               0
            Conv2d-9           [-1, 16, 10, 10]           2,320
      BatchNorm2d-10           [-1, 16, 10, 10]              32
          Dropout-11           [-1, 16, 10, 10]               0
           Conv2d-12             [-1, 16, 8, 8]           2,320
      BatchNorm2d-13             [-1, 16, 8, 8]              32
          Dropout-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0512, Accuracy: 9834/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0357, Accuracy: 9885/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0191, Accuracy: 9946/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0201, Accuracy: 9939/10000 (99%)



loss=0.00412730872631073 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 34.99it/s]



Test set: Average loss: 0.0204, Accuracy: 9941/10000 (99%)

